In [1]:
# Adding the Aima-Python folder to the folder

import sys
sys.path.insert(1, 'D:/Masters/Knowledge Representation/workplace/aima-python')

In [8]:
from agents import *
from search import *
from notebook import psource

In [3]:
"""
Represents the initial state and goal state for the Missionaries and Cannibals problem.
"""

INITIAL_STATE = (3, 3, 1)
GOAL_STATE = (0, 0, 0)


In [6]:
import operator


def add_tuples(a, b):
    return operate_on_tuples(a, b, operator.add)


def subtract_tuples(a, b):
    return operate_on_tuples(a, b, operator.sub)


def operate_on_tuples(a, b, operation):
    return tuple(map(operation, a, b))


def contains_negative(collection):
    """Return True if any negative value exists in the collection."""
    return any(n < 0 for n in collection)

In [7]:
import operator



class State:
    """Represents the state in the Missionaries and Cannibals problem."""

    def __init__(self, missionaries, cannibals, boat):
        self.value = (missionaries, cannibals, boat)
        self.missionaries = missionaries
        self.cannibals = cannibals
        self.boat = boat

    @classmethod
    def value_of(cls, state):
        if not cls.__is_valid_tuple(state):
            raise ValueError(str(state) + " must be a tuple with 3 elements.")
        return State(state[0], state[1], state[2])

    def is_valid(self):
        if contains_negative(self.value):
            return False
        elif self.__has_more_than_one_boat():
            return False
        elif self.__has_more_cannibals_than_missionaries():
            return False
        elif self.value > INITIAL_STATE:
            return False
        else:
            return True

    def __has_more_than_one_boat(self):
        return self.boat > 1

    def __has_more_cannibals_than_missionaries(self):
        return self.__more_cannibals_on_wrong_side() or self.__more_cannibals_on_right_side()

    def __more_cannibals_on_wrong_side(self):
        """Return True when more cannibals than missionaries exist on the wrong side of the river."""
        return ((self.missionaries == 1 and self.cannibals == 3) or
                (self.missionaries == 2 and self.cannibals == 3) or
                (self.missionaries == 1 and self.cannibals == 2))

    def __more_cannibals_on_right_side(self):
        """Return True when more cannibals than missionaries exist on the right side of the river.
        The "right" side of the river is the side opposite of the starting side.
        """
        return ((self.missionaries == 2 and self.cannibals == 1) or
                (self.missionaries == 1 and self.cannibals == 0))

    def __add__(self, other):
        result = self.__operate(other, operator.add)
        return self.value_of(result)

    def __sub__(self, other):
        result = self.__operate(other, operator.sub)
        return self.value_of(result)

    def __operate(self, other, operation):
        if self.__is_valid_tuple(other):
            return operate_on_tuples(self.value, other, operation)
        elif isinstance(other, State):
            return operate_on_tuples(self.value, other.value, operation)
        else:
            raise ValueError(self.__get_invalid_operand_error(other))

    @staticmethod
    def __is_valid_tuple(other):
        return isinstance(other, tuple) and len(other) == 3

    @staticmethod
    def __get_invalid_operand_error(other):
        return str(other) + " must be an instance of State or a tuple of length 3."

    def __repr__(self):
        return '<State {}>'.format(self.value)

    def __str__(self):
        return '<State {}>'.format(self.value)

    def __lt__(self, other):
        self.__ensure_instance_of_state(other)
        return self.value < other.value

    def __eq__(self, other):
        return isinstance(other, State) and self.value == other.value

    def __hash__(self):
        return hash(self.value)

    @staticmethod
    def __ensure_instance_of_state(other):
        if not isinstance(other, State):
            raise ValueError(str(other) + " must be an instance of State")

In [9]:
class MissionariesAndCannibals(Problem):

    def __init__(self):
        initial_state = State.value_of(INITIAL_STATE)
        goal_state = State.value_of(GOAL_STATE)
        super().__init__(initial_state, goal_state)

    def actions(self, state):
        all_actions = self.get_all_actions()
        return self.get_valid_actions(state, all_actions)

    @staticmethod
    def get_all_actions():
        return {
            (1, 0, 1),
            (2, 0, 1),
            (0, 1, 1),
            (0, 2, 1),
            (1, 1, 1)
        }

    def get_valid_actions(self, state, all_actions):
        is_action_valid_lambda = self.get_is_action_valid_lambda(state)
        return set(filter(is_action_valid_lambda, all_actions))

    def get_is_action_valid_lambda(self, state):
        return lambda action: self.is_action_valid(state, action)

    def is_action_valid(self, state, action):
        operate = self.get_operation(state.boat)
        result = operate(state, action)

        return result.is_valid()

    def result(self, state, action):
        operate = self.get_operation(state.boat)
        return operate(state, action)

    @staticmethod
    def get_operation(boat):
        """Subtract action from state if boat is on initial side of river."""
        return operator.sub if boat == 1 else operator.add

In [10]:
def print_path(path):
    for node in path:
        print(node.state.value)

In [20]:
problem = MissionariesAndCannibals()
result = iterative_deepening_search(problem)
print_path(result.path())

(3, 3, 1)
(3, 1, 0)
(3, 2, 1)
(3, 0, 0)
(3, 1, 1)
(2, 0, 0)
(2, 2, 1)
(0, 2, 0)
(0, 3, 1)
(0, 1, 0)
(0, 2, 1)
(0, 0, 0)
